In [ ]:
import chromadb

client = chromadb.PersistentClient(path="chroma-test-compact")


In [ ]:
from chromadb.segment.impl.vector.local_persistent_hnsw import PersistentData
from chromadb.segment import VectorReader
from chromadb.types import SegmentScope, Operation

# client._server._manager
vector_segments = [s for s in client._server._sysdb.get_segments() if s['scope'] == SegmentScope.VECTOR]

for s in vector_segments:
    col=client._server._get_collection(s['collection'])
    client._server._manager.hint_use_collection(s['collection'], Operation.ADD)
    segment=client._server._manager.get_segment(s['collection'], VectorReader)
    segment._apply_batch(segment._curr_batch)
    segment._persist()
    metadata = PersistentData.load_from_file(
        f"chroma-test-compact/{s['id']}/index_metadata.pickle")
    print(len(metadata.id_to_label),col['name'] , metadata.max_seq_id)
    segment.close_persistent_index()

In [ ]:
import chromadb

client = chromadb.PersistentClient(path="chroma-test-compact")

client.list_collections()

col = client.get_or_create_collection("chroma-qna7")

res= col.get(include=["embeddings"])
print(len(res["ids"]),len(res["embeddings"]))